In [ ]:
# Mount Google Drive
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
DATA_LOC = "/content/drive/MyDrive/Rhexis/datasets/test_pulls"
REPO_LOC = "/content/drive/MyDrive/Trajectories/rhexis-trajectory"

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

sys.path.insert(0, f"{REPO_LOC}/Trajectory_Classification")
from utils import *

%load_ext autoreload
%autoreload 2

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression(random_state=0, max_iter=1000)
param_grid = {
    "pca__n_components": list(range(3, 20)),
}
bin_grid = list(range(5, 25))
results_lr = grid_search_with_bins(make_custom_pipeline(clf_lr, True), param_grid, bin_grid)

In [ ]:
results_lr["best_score"]

In [ ]:
results_lr["best_params"]

In [ ]:
results_lr["best_search"].score(results_lr["X_test"], results_lr["y_test"])

## Quadratic Gaussian Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf_qda = QuadraticDiscriminantAnalysis()
param_grid = {
    "pca__n_components": list(range(3, 20)),
    "quadraticdiscriminantanalysis__reg_param": [
        1e-6,
        1e-3,
        1e-2,
        1e-1,
        2e-1,
        3e-1,
        4e-1,
        5e-1,
    ],
}
bin_grid = list(range(5, 25))
results_qda = grid_search_with_bins(clf_qda, param_grid, bin_grid)

In [ ]:
results_qda["best_score"]

In [ ]:
results_qda["best_params"]

In [ ]:
results_qda["best_search"].score(results_qda["X_test"], results_qda["y_test"])

## Multi-Layer Perceptron

From above experiments, we are sure that 21 bins is the optimal amount, and do not need to grid search over that parameter any further.

In [ ]:
from sklearn.neural_network import MLPClassifier

X_train, X_test, y_train, y_test = get_data_for_fixed_bins(21)

clf_mlp = MLPClassifier(
    solver="adam", alpha=1e-5, hidden_layer_sizes=(1000, 1000, 1000), random_state=1
)
param_grid = {
    "pca__n_components": list(range(10, 15)),
    "mlpclassifier__learning_rate_init": [1e-9, 1e-6, 1e-3, 1e-2, 1e-1],
}
search = grid_search(clf_mlp, param_grid, X_train, y_train)

In [ ]:
search.best_score_

In [ ]:
search.best_params_

In [ ]:
search.score(X_test, y_test)

## Histogram-based Gradient Boosting (LightGBM)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

X_train, X_test, y_train, y_test = get_data_for_fixed_bins(21)

clf_hgbc = HistGradientBoostingClassifier()
param_grid = {
    "pca__n_components": [10],
    "histgradientboostingclassifier__loss": ['log_loss', 'binary_crossentropy'],
    "histgradientboostingclassifier__learning_rate": [1e-9, 1e-6, 1e-3, 1e-2, 1e-1],
    "histgradientboostingclassifier__min_samples_leaf": list(range(5, 20)),
    "histgradientboostingclassifier__l2_regularization": [0, 1e-9, 1e-6, 1e-3, 1e-2, 0.1, 0.2, 0.3, 0.5, 0.75]
    
}
hgbc_search = grid_search(clf_hgbc, param_grid, X_train, y_train)

In [ ]:
hgbc_search.best_score_

In [ ]:
hgbc_search.best_params_

In [ ]:
hgbc_search.score(X_test, y_test)

## Generate ROC Curves for Logistic Regression, Quadratic GDA, and Multilayer Perceptron, HBGB

In [ ]:
from sklearn.metrics import RocCurveDisplay

# Logistic Regression
lr_best_n_bins = results_lr['best_params']['data__n_bins']
best_clf_lr = results_lr['results'][lr_best_n_bins].best_estimator_
fig = RocCurveDisplay.from_estimator(best_clf_lr, results_lr['X_test'], results_lr['y_test'], name="Logistic Regression")

# Quadratic GDA
qda_best_n_bins = results_qda['best_params']['data__n_bins']
best_clf_qda = results_qda['results'][qda_best_n_bins].best_estimator_
RocCurveDisplay.from_estimator(best_clf_qda, results_qda['X_test'], results_qda['y_test'], name="Quadratic GDA", ax=fig.ax_)

# Multilayer Perceptron
best_clf_mlp = search.best_estimator_
RocCurveDisplay.from_estimator(best_clf_mlp, X_test, y_test, name="Multilayer Perceptron", ax=fig.ax_)

# Histogram based Gradient Boosting
best_clf_hbgc = hgbc_search.best_estimator_
RocCurveDisplay.from_estimator(best_clf_hbgc, X_test, y_test, name="Gradient Boosting", ax=fig.ax_)

plt.title("ROC Curves for LR, Quadratic GDA, MLP, HBGB")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.savefig('ROC_Curves.pdf', dpi=120)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier()
clf_rf.fit(X_train, y_train)
clf_rf.score(X_test, y_test)
